<a href="https://colab.research.google.com/github/farroshsy/Classification_BigData/blob/main/ClassificationonDatabricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Big Data Assignment for Classification (in Class)**
## Use Apache Spark MLlib on Databricks
---
Name: Farros Hilmi Syafei 
<br>
Student ID: 5025201012
<br>
Class: Big Data A
<br>
Lecturer: Abdul Munif, S.Kom., M.Sc.


## Source:
1. https://docs.databricks.com/machine-learning/train-model/mllib/index.html

In [22]:
!java --version
!python --version

openjdk 11.0.19 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Python 3.10.11


## Installing Apache Spark (PySpark)

In [23]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Initialize Apache Spark context

In [24]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Check spark session
print(spark)

# Dataset Review

The Adult dataset is publicly available at the UCI Machine Learning Repository. This data derives from census data and consists of information about 48842 individuals and their annual income. You can use this information to predict if an individual earns <=50K or >50k a year. The dataset consists of both numeric and categorical variables.

Attribute Information:
*   age: continuous
*   workclass: Private,Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked
*   fnlwgt: continuous
*   education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc...
*   education-num: continuous
*   marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent...
*   occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners...
*   relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried
*   race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black
*   sex: Female, Male
*   capital-gain: continuous
*   capital-loss: continuous
*   hours-per-week: continuous
*   native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany...

Target/Label: - <=50K, >50K




# Load Data

In [25]:
from pyspark.sql.types import DoubleType, StringType, StructField, StructType
 
schema = StructType([
  StructField("age", DoubleType(), False),
  StructField("workclass", StringType(), False),
  StructField("fnlwgt", DoubleType(), False),
  StructField("education", StringType(), False),
  StructField("education_num", DoubleType(), False),
  StructField("marital_status", StringType(), False),
  StructField("occupation", StringType(), False),
  StructField("relationship", StringType(), False),
  StructField("race", StringType(), False),
  StructField("sex", StringType(), False),
  StructField("capital_gain", DoubleType(), False),
  StructField("capital_loss", DoubleType(), False),
  StructField("hours_per_week", DoubleType(), False),
  StructField("native_country", StringType(), False),
  StructField("income", StringType(), False)
])


In [26]:
dataset = spark.read.format("csv").schema(schema).load("adult.data")
cols = dataset.columns

In [27]:
dataset.show()

+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
| age|        workclass|  fnlwgt|    education|education_num|      marital_status|        occupation|  relationship|               race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|
+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
|39.0|        State-gov| 77516.0|    Bachelors|         13.0|       Never-married|      Adm-clerical| Not-in-family|              White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|
|50.0| Self-emp-not-inc| 83311.0|    Bachelors|         13.0|  Married-civ-spouse|   Exec-managerial|       Husband|              White|   Male|         0.0|         0.

In [28]:
display(dataset)

DataFrame[age: double, workclass: string, fnlwgt: double, education: string, education_num: double, marital_status: string, occupation: string, relationship: string, race: string, sex: string, capital_gain: double, capital_loss: double, hours_per_week: double, native_country: string, income: string]

# Preprocess Data

To use algorithms like Logistic Regression, you must first convert the categorical variables in the dataset into numeric variables. There are two ways to do this.

Category Indexing

This is basically assigning a numeric value to each category from {0, 1, 2, ...numCategories-1}. This introduces an implicit ordering among your categories, and is more suitable for ordinal variables (eg: Poor: 0, Average: 1, Good: 2)

One-Hot Encoding

This converts categories into binary vectors with at most one nonzero value (eg: (Blue: [1, 0]), (Green: [0, 1]), (Red: [0, 0]))

This notebook uses a combination of StringIndexer and, depending on your Spark version, either OneHotEncoder or OneHotEncoderEstimator to convert the categorical variables. OneHotEncoder and OneHotEncoderEstimator return a SparseVector.

Since there is more than one stage of feature transformations, use a Pipeline to tie the stages together. This simplifies the code.

In [29]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.feature import OneHotEncoder
  
categoricalColumns = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country"]


In [30]:
stages = [] # stages in Pipeline

for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]

In [31]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol="income", outputCol="label")
stages += [label_stringIdx]

In [32]:
# Transform all features into a vector using VectorAssembler
numericCols = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [33]:
stages

[StringIndexer_62289df383ec,
 OneHotEncoder_c18bd8e38e6d,
 StringIndexer_13ae27b656d4,
 OneHotEncoder_cbab3e8d62ac,
 StringIndexer_67bfb496de3d,
 OneHotEncoder_65ed5463cd12,
 StringIndexer_369f9fd8c459,
 OneHotEncoder_c7d2d6be6761,
 StringIndexer_3819326c0b32,
 OneHotEncoder_a70804aa3e94,
 StringIndexer_6a8fc4085a44,
 OneHotEncoder_e806e2c84f75,
 StringIndexer_c432d0e7abc0,
 OneHotEncoder_9fbceeaf5475,
 StringIndexer_78108b25e285,
 OneHotEncoder_a8a895e19e82,
 StringIndexer_1d79fe210d0b,
 VectorAssembler_1d5c5b963460]

In [34]:
from pyspark.ml.classification import LogisticRegression
  
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(dataset)
preppedDataDF = pipelineModel.transform(dataset)

In [35]:
preppedDataDF.show()

+----+-----------------+--------+-------------+-------------+--------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+--------------+-----------------+--------------+-----------------+-------------------+----------------------+---------------+------------------+-----------------+--------------------+---------+-------------+--------+-------------+-------------------+----------------------+-----+--------------------+
| age|        workclass|  fnlwgt|    education|education_num|      marital_status|        occupation|  relationship|               race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income|workclassIndex|workclassclassVec|educationIndex|educationclassVec|marital_statusIndex|marital_statusclassVec|occupationIndex|occupationclassVec|relationshipIndex|relationshipclassVec|raceIndex| raceclassVec|sexIndex|  sexclassVec|native_countryIndex|native_countryclassVec|label|      

In [36]:
# Fit model to prepped data
lrModel = LogisticRegression().fit(preppedDataDF)
 
# ROC for training data
display(lrModel, preppedDataDF, "ROC")

LogisticRegressionModel: uid=LogisticRegression_98bd0b02ddd5, numClasses=2, numFeatures=100

DataFrame[age: double, workclass: string, fnlwgt: double, education: string, education_num: double, marital_status: string, occupation: string, relationship: string, race: string, sex: string, capital_gain: double, capital_loss: double, hours_per_week: double, native_country: string, income: string, workclassIndex: double, workclassclassVec: vector, educationIndex: double, educationclassVec: vector, marital_statusIndex: double, marital_statusclassVec: vector, occupationIndex: double, occupationclassVec: vector, relationshipIndex: double, relationshipclassVec: vector, raceIndex: double, raceclassVec: vector, sexIndex: double, sexclassVec: vector, native_countryIndex: double, native_countryclassVec: vector, label: double, features: vector]

'ROC'

In [37]:
display(lrModel, preppedDataDF)

LogisticRegressionModel: uid=LogisticRegression_98bd0b02ddd5, numClasses=2, numFeatures=100

DataFrame[age: double, workclass: string, fnlwgt: double, education: string, education_num: double, marital_status: string, occupation: string, relationship: string, race: string, sex: string, capital_gain: double, capital_loss: double, hours_per_week: double, native_country: string, income: string, workclassIndex: double, workclassclassVec: vector, educationIndex: double, educationclassVec: vector, marital_statusIndex: double, marital_statusclassVec: vector, occupationIndex: double, occupationclassVec: vector, relationshipIndex: double, relationshipclassVec: vector, raceIndex: double, raceclassVec: vector, sexIndex: double, sexclassVec: vector, native_countryIndex: double, native_countryclassVec: vector, label: double, features: vector]

In [ ]:
# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = preppedDataDF.select(selectedcols)
display(dataset)
dataset.show()

In [42]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())

22832
9729


# Fit and Evaluate Models
Now, try out some of the Binary Classification algorithms available in the Pipelines API.

Out of these algorithms, the below are also capable of supporting multiclass classification with the Python API:

Decision Tree Classifier
Random Forest Classifier
These are the general steps to build the models:

Create initial model using the training set
Tune parameters with a ParamGrid and 5-fold Cross Validation
Evaluate the best model obtained from the Cross Validation using the test set
Use the BinaryClassificationEvaluator to evaluate the models, which uses areaUnderROC as the default metric.

# Logistic Regression

In the Pipelines API, you can now perform Elastic-Net Regularization with Logistic Regression, as well as other linear methods.

In [54]:
from pyspark.ml.classification import LogisticRegression
 
# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)
 
# Train model with Training Data
lrModel = lr.fit(trainingData)

By adding regParam=0.3, elasticNetParam=0.8, the result of the evaluator become worse which is 0.5. So here I didnt add it to get the better result

In [55]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)

In [56]:
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
selected.show()

+-----+----------+--------------------+----+---------------+
|label|prediction|         probability| age|     occupation|
+-----+----------+--------------------+----+---------------+
|  0.0|       1.0|[0.16646708724833...|36.0| Prof-specialty|
|  0.0|       0.0|[0.70012031106347...|32.0| Prof-specialty|
|  0.0|       0.0|[0.53453682932965...|33.0| Prof-specialty|
|  0.0|       0.0|[0.67559303222185...|39.0| Prof-specialty|
|  0.0|       0.0|[0.62237992652147...|39.0| Prof-specialty|
|  0.0|       0.0|[0.61047384917734...|50.0| Prof-specialty|
|  0.0|       0.0|[0.60270631574160...|51.0| Prof-specialty|
|  0.0|       0.0|[0.60470503143805...|60.0| Prof-specialty|
|  0.0|       0.0|[0.74230866284291...|34.0| Prof-specialty|
|  0.0|       0.0|[0.98954486365582...|20.0| Prof-specialty|
|  0.0|       1.0|[0.35443820573118...|35.0| Prof-specialty|
|  0.0|       0.0|[0.52012311192879...|42.0| Prof-specialty|
|  0.0|       0.0|[0.57001207297900...|43.0| Prof-specialty|
|  0.0|       0.0|[0.724

In [57]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
 
# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.9010633018702178

In [58]:
evaluator.getMetricName()

'areaUnderROC'

In [59]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The

In [60]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
 
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [61]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
 
# Run cross validations
cvModel = cv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

In [62]:
# Use the test set to measure the accuracy of the model on new data
predictions = cvModel.transform(testData)

In [63]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

0.8997203469558716

In [64]:
print('Model Intercept: ', cvModel.bestModel.intercept)

Model Intercept:  -6.336365999298099


In [67]:
print(cvModel.bestModel.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0, current: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained opt

In [70]:
weights = cvModel.bestModel.coefficients
weights = [(float(w),) for w in weights]  # convert numpy type to float, and to tuple
weightsDF = spark.createDataFrame(weights, ["Feature Weight"])
weightsDF.show()

+--------------------+
|      Feature Weight|
+--------------------+
| 0.10492555077522134|
| -0.3128222519796268|
|-0.06075451360468...|
| -0.2721306281794785|
|-0.14581243598919214|
|  0.3495304348424823|
| 0.47753854084743574|
| -2.4920423132828398|
|-0.18817807301492645|
| 0.02457242800190841|
|  0.4055326717725701|
|  0.6041914661074328|
| 0.09249180451745206|
| -0.6495393362369409|
|-0.06796598188360077|
|   -0.50113851942221|
| -0.7555258791036417|
|  0.8758631574746794|
| -0.6424968866720012|
| -0.3124342654495672|
+--------------------+
only showing top 20 rows



In [71]:
# View best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
selected.show()

+-----+----------+--------------------+----+---------------+
|label|prediction|         probability| age|     occupation|
+-----+----------+--------------------+----+---------------+
|  0.0|       1.0|[0.21060910009532...|36.0| Prof-specialty|
|  0.0|       0.0|[0.68097448822474...|32.0| Prof-specialty|
|  0.0|       0.0|[0.54341491059548...|33.0| Prof-specialty|
|  0.0|       0.0|[0.65860417846993...|39.0| Prof-specialty|
|  0.0|       0.0|[0.61463593853443...|39.0| Prof-specialty|
|  0.0|       0.0|[0.60241279947003...|50.0| Prof-specialty|
|  0.0|       0.0|[0.59586897256623...|51.0| Prof-specialty|
|  0.0|       0.0|[0.59681289807548...|60.0| Prof-specialty|
|  0.0|       0.0|[0.72170646267586...|34.0| Prof-specialty|
|  0.0|       0.0|[0.96796930272851...|20.0| Prof-specialty|
|  0.0|       1.0|[0.47756430733627...|35.0| Prof-specialty|
|  0.0|       0.0|[0.52627860330667...|42.0| Prof-specialty|
|  0.0|       0.0|[0.56664870040475...|43.0| Prof-specialty|
|  0.0|       0.0|[0.696

# Decision Trees

The Decision Trees algorithm is popular because it handles categorical data and works out of the box with multiclass classification tasks.

In [72]:
from pyspark.ml.classification import DecisionTreeClassifier
 
# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)
 
# Train model with Training Data
dtModel = dt.fit(trainingData)

In [81]:
print("numNodes = ", dtModel.numNodes)
print("depth = ", dtModel.depth)

numNodes =  11
depth =  3


In [77]:
display(dtModel)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_13d04a02294f, depth=3, numNodes=11, numClasses=2, numFeatures=100

In [74]:
# Make predictions on test data using the Transformer.transform() method.
predictions = dtModel.transform(testData)

In [75]:
predictions.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- age: double (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [78]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
display(selected)

DataFrame[label: double, prediction: double, probability: vector, age: double, occupation: string]

In [79]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.7309739438488391

In [80]:
dt.getImpurity()

'gini'

In [82]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1, 2, 6, 10])
             .addGrid(dt.maxBins, [20, 40, 80])
             .build())

In [83]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
 
# Run cross validations
cvModel = cv.fit(trainingData)
# Takes ~5 minutes

In [84]:
print("numNodes = ", cvModel.bestModel.numNodes)
print("depth = ", cvModel.bestModel.depth)


numNodes =  389
depth =  10


In [85]:
# Use test set to measure the accuracy of the model on new data
predictions = cvModel.transform(testData)

In [86]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

0.7882520737057215

In [88]:
# View Best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
display(selected)

DataFrame[label: double, prediction: double, probability: vector, age: double, occupation: string]

# Random Forest


Random Forests uses an ensemble of trees to improve model accuracy.

In [89]:
from pyspark.ml.classification import RandomForestClassifier
 
# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")
 
# Train model with Training Data
rfModel = rf.fit(trainingData)

In [90]:
# Make predictions on test data using the Transformer.transform() method.
predictions = rfModel.transform(testData)

In [91]:
predictions.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- age: double (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income: string (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [92]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
display(selected)

DataFrame[label: double, prediction: double, probability: vector, age: double, occupation: string]

In [93]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
 
# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.8830641335350443

In [94]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
 
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())

In [95]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
 
# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(trainingData)

In [96]:
# Use the test set to measure the accuracy of the model on new data
predictions = cvModel.transform(testData)

In [97]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

0.890665079151747

In [98]:
# View Best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
display(selected)

DataFrame[label: double, prediction: double, probability: vector, age: double, occupation: string]

# Make Predictions
As Random Forest gives the best areaUnderROC value, use the bestModel obtained from Random Forest for deployment, and use it to generate predictions on new data. Instead of new data, this example generates predictions on the entire dataset.



In [99]:
bestModel = cvModel.bestModel

In [100]:
# Generate predictions for entire dataset
finalPredictions = bestModel.transform(dataset)

In [101]:
# Evaluate best model
evaluator.evaluate(finalPredictions)

0.8972959639997035

In [102]:
finalPredictions.createOrReplaceTempView("finalPredictions")